<a href="https://colab.research.google.com/github/mbogantesv/PHC_site_graphics/blob/master/Limpieza_Datos_Echandi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import files


In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [7]:
raw_db=pd.read_excel(list(uploaded.keys())[0], skiprows = [0], sheet_name= "Output Final")

raw_db = raw_db.dropna(thresh=25)
for i in range(len(raw_db["Test"])):
  raw_db["Test"][i] = str(raw_db["Test"][i])[:-1]

raw_db = raw_db.drop_duplicates(subset=["MRF Number"])
raw_db = raw_db.dropna(axis = 0, subset=["Result (Comma Separated)"])
raw_db = raw_db.reset_index()

def cmpa2(small,large):
  lis=[]
  resultw = []
  for i in range(len(large)):
    lis.append(large[i] in small)
  for j in lis:
    if j == True:
      resultw.append("Positivo")
    else:
      resultw.append("Negativo")
  return resultw

keep_columns = ['MRF Number', 'Country', 'Indication', 'Tissue Type',
       'Block Number(s)', 'Initials', 'MRF Date (collection)', 'Pick-up Date',
       'Result Date', 'Institution', 'Treatment Doctor']

db_columns = keep_columns + ["Test"] + ["Result"]

contador = 0
for i in raw_db["Test"]:
  contador=contador + i.count(",")
clean_db = pd.DataFrame([[0] * len(db_columns)] * contador,columns = db_columns)

df_Positives = raw_db.iloc[:,21:38]
df_Tests = raw_db.iloc[:,40:61]

count =0
for i in range(len(raw_db)):
  done_tests = raw_db["Test"][i].split(",")
  a= list((df_Positives.iloc[i] == 1).to_numpy().nonzero()[0]) 
  b= list((df_Tests.iloc[i] == 1).to_numpy().nonzero()[0])
  compare = cmpa2(a,b)
  for j in range(len(done_tests)):
    for k in keep_columns:
      clean_db[k][count]=raw_db[k][i]
    clean_db["Test"][count] = done_tests[j]
    try:
      clean_db["Result"][count]= str(compare[j])
    except IndexError:
      clean_db["Result"][count] = "ERROR"
    count = count+1

clean_db.to_csv("Datos_limpios.csv", index = False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5

In [0]:
files.download("Datos_limpios.csv")